In [2]:
import SimpleITK as sitk
import os
from dltk.io.augmentation import *
from dltk.io.preprocessing import *
import tensorflow as tf
import nibabel as nib



C:\Users\Nakyilkim\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
root = 'D:\segmentiation/'
ct_set = os.path.join(root,'ct_train_test/ct_train/')
mr_set = os.path.join(root,'mr_train_test/mr_train/')

name_list = []
filenames = os.listdir(ct_set)

for i in filenames:
    if i[:13] not in name_list:
        name_list.append(i[:13])
        
total_list = []
for i in name_list:
    temp_name = i
    temp_image = i+'_image.nii'
    temp_label = i+'_label.nii'
    total_list.append({'name' : temp_name,'image':temp_image,'label':temp_label})

In [4]:
def pad3d(array):
    
    assert len(array.shape) == 3
    
    height = array.shape[0]
    depth = array.shape[2]
    
    print("Array Shape", array.shape)
    print("height, Depth",height,depth)
    
    if (height - depth) % 2 :
        pad_front = int((height + 1 - depth) / 2)
        pad_back = int((height - 1 - depth) / 2)
    else:
        pad_front = pad_back = int((height - depth) / 2)
        
    print("pad_front, pad_back",pad_front,pad_back)
    
    npad = ((0,0),(0,0),(pad_front,pad_back))
    array_padding = np.pad(array, npad, 'constant', constant_values=(0))
    array_padding[array_padding<0] = 0
    
    return array_padding


def image_preprocess(image, new_size, mask=False):
    assert np.sum(image.shape==image.shape[0])!=3    
    
    ratio = new_size / image.shape[0]
    
    image = scipy.ndimage.zoom(image, zoom=ratio, order=0)

    if mask:
        channel = 7 + 1 #background
        image = image.reshape(-1)
        image = label_encoder.fit_transform(image)
        
        
        print("annotation shape",image.shape)
        print("unique value", np.unique(image))
        
        
        image = to_categorical(image, class_num)
        print("processed mask shape",image.shape)

        
    else:
        channel = 1
    # reshape to raw shape
    image = image.reshape((new_size,)*3 + (channel,))
    print("Reshaped shape",image.shape)
    
    return image

def nii_loader(path,file_name):
    file_path =  os.path.join(path,file_name)
    fn = os.listdir(file_path)
    image = (nib.load(file_path + '/' + fn[0]))
    return image



In [5]:
## Making Read_fn

data_path = ct_set

def read_fn(file_reference, mode, new_size=256 ,params=None):
    
    
    ## file_refrences를 파일 path로 일단 생각하자
    
    for file_dict in file_reference:
        
        file_name = file_dict['name']
        file_image = file_dict['image']
        file_label = file_dict['label']
        
        
        ## 이미지 데이터를 로드하는 함수 인듯?

        
        t1 = nii_loader(data_path,file_image)
        lbl = nii_loader(data_path, file_label)
        
        
        ## whitening , 평균 0, 표준편차 1로 만들자
        
        ###### 나중에 추가하자!
        #t1 = whitening(t1)
        
        ## 재혁이 전처리
        
        print("="*10,t1.shape)
        
        t1 = pad3d(t1)
        t1 = image_preprocess(t1,new_size=new_size)
        
        
        # 4D Tensor with a dummy dimension for channels
        # ex (363, 512, 512) --> (362, 512, 512, 1)
        t1 = t1[...,np.newaxis]
        
        ### 
        print("Preprocessed Image Shape", t1.shape)
        
        
        ## prdeict mode 일 때는 label이 없다! 따라서 label 없는 값을 yield!
        if mode == tf.estimator.ModeKeys.PREDICT:
            yield{'features': {'x': t1},
                 'metadata':{
                     'subject_id': subgject_id,
                     'sitk': sitk_t1
                 }}
            
            
        ## Labels:

        
                ## 이미지 데이터를 로드하는 함수 인듯?

        #  lbl = image_preprocess(lbl,new_size=new_size,mask=True)

        lbl = image_preprocess(lbl,new_size=new_size)
        lbl = lbl[..., np.newaxis]
        y = lbl
        
        print("Preprocessed label Shape", y.shape)

        
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            
            # Insert augmentation here
            pass
        
#         if params['extract_examples']:
#             images = extract_random_example_array(
#                 t1,
#                 example_size=params['example_size'],
#                 n_examples=params['n_examples'])
            
#             # Loop the extracted image patches and yield
#             for e in range(params['n_examples']):
#                 yield {'features': {'x': images[e].astype(np.float32)},
#                        'labels': {'y': y.astype(np.int32)},
#                        'metadata': {
#                            'subject_id': subject_id,
#                            'sitk': sitk_t1}}
        
    
        ## Batch 1 이라고 할 때
#         else:
        yield {'features' : {'x': t1},
                'labels':{'y' : y}}
        

In [6]:
reader_params = {'n_examples': 1,
                 'example_size': [256, 256, 256],
                 'extract_examples': True}

reader_example_shapes = {'features': {'x': reader_params['example_size'] + [1,]},
                         'labels': {'y': [256,256,256] + [1,] }}


reader_example_dtypes = {'features': {'x': tf.float32},
                         'labels': {'y': tf.int32}}


from dltk.io.abstract_reader import Reader
reader = Reader(read_fn=read_fn,
               dtypes=reader_example_dtypes)

In [7]:
input_fn, qinit_hook = reader.get_inputs(total_list,
                                        tf.estimator.ModeKeys.TRAIN,
                                        example_shapes=reader_example_shapes,
                                        batch_size=1,
                                        shuffle_cache_size=1,
                                        params=reader_params)


features, labels = input_fn()

In [8]:
labels

{'y': <tf.Tensor 'IteratorGetNext:1' shape=(?, 256, 256, 256, 1) dtype=int32>}

## Model Function

In [9]:
from matplotlib import pyplot as plt
from IPython import display

%matplotlib inline


class NotebookLoggingHook(tf.train.SessionRunHook):
    def __init__(self, fetches):
        self.fetches = fetches
        self.loss = []
        
    def before_run(self, run_context):
        # Add the tensors to fetch to the session
        return tf.train.SessionRunArgs(self.fetches)
    
    def after_run(self, run_context, run_values):
        # Extract the results of the fetched tensors
        fetch_dict = run_values.results
        
        # Assume to have {'loss': scalar, 'input': img, 'output': img, 'truth': img}
        self.loss.append(fetch_dict['loss'])
        
        # Plot stuff using `matplotlib`
        f, axarr = plt.subplots(2, 2, figsize=(16,8))
        axarr[0,0].imshow(np.squeeze(fetch_dict['x'][0,0,:,:,0]), cmap='gray')
        axarr[0,0].set_title('Input: x')
        axarr[0,0].axis('off')

        axarr[0,1].plot(self.loss)
        axarr[0,1].set_title('Crossentropy loss')
        axarr[0,1].set_yscale('log')
        axarr[0,1].axis('on')

        axarr[1,0].imshow(np.squeeze(fetch_dict['y_'][0,0,:,:]), cmap='gray', vmin=0, vmax=1)
        axarr[1,0].set_title('Prediction: y_')
        axarr[1,0].axis('off')

        axarr[1,1].imshow(np.squeeze(fetch_dict['y'][0,0,:,:]), cmap='gray', vmin=0, vmax=1)
        axarr[1,1].set_title('Truth: y')
        axarr[1,1].axis('off')

        display.clear_output(wait=True)
        display.display(plt.gcf())
        plt.close(f)

In [10]:
# Create a `NotebookLoggingHook`
nl_hook = NotebookLoggingHook(None)
hooks = [nl_hook]


## Model function 만들기 
# 인자 : Features, labels,mode, params
def model_fn(features, labels, mode, params):
    
     # 인풋을 넣으면 자동으로 트레인을 할 수 있도록 틀을 짜놓고
     # 모델은 바꾸면서 낄 수 있도록
#     residual_fcn_3d(inputs,
#                     num_classes,
#                     num_res_units=1,
#                     filters=(16, 32, 64, 128),
#                     strides=((1, 1, 1), (2, 2, 2), (2, 2, 2), (2, 2, 2)),
#                     mode=tf.estimator.ModeKeys.EVAL,
#                     use_bias=False,
#                     activation=tf.nn.relu6,
#                     kernel_initializer=tf.initializers.variance_scaling(distribution='uniform'),
#                     bias_initializer=tf.zeros_initializer(),
#                     kernel_regularizer=None,
#                     bias_regularizer=None)
    # 예시로 모델 하나 불러오기
    from dltk.networks.segmentation.fcn import residual_fcn_3d
    
    ### 왜 Strides 가 (1,2,2) 일까...
    ### Z 축으로 이동할 때는 한 칸씩 들어 간다는 뜻인가
    net_output_ops = residual_fcn_3d(features['x'], 8, num_res_units=1, filters=(16, 32, 64),
                                    strides=((2, 2, 2), (2, 2, 2),  (2, 2, 2)), mode = mode)
#     net_output_ops = residual_fcn_3d(features['x'], 8, num_res_units=1, filters=(16, 32, 64),
#                                     strides=((1, 1, 1), (1, 2, 2),  (, 2, 2)), mode = mode)

    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions=net_output_ops,
            export_outputs={'out':tf.estimator.export.PredictOutput(net_output_ops)})
    
    ## sparse soft max???
    loss = tf.losses.sparse_softmax_cross_entropy(labels['y'],
                                                 net_output_ops['logits'])
    ## global step ??
    global_step = tf.train.get_global_step()
    optimizer = tf.train.AdamOptimizer(learning_rate=params["learning_rate"],
                                      epsilon=1e-5)
    
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_op = optimizer.minimize(loss, global_step)
    
    ## 노트북에 플랏팅하기
    
    my_notebook_fetches = {}
    my_notebook_fetches['loss'] = loss
    my_notebook_fetches['x'] = features['x']
    my_notebook_fetches['y'] = labels['y']
    my_notebook_fetches['y_'] = net_output_ops['y_']
    nl_hook.fetches = my_notebook_fetches

    
    return tf.estimator.EstimatorSpec(mode=mode,
                                     predictions=net_output_ops,
                                     loss=loss,
                                     train_op=train_op,
                                     eval_metric_ops=None)
    

### 에러 발생
1. 튜토리얼 데로라면 원핫이 안 필요한듯
2. 튜토리얼은 input shape 를 앞 뒤로 더 해줌/ 앞에는 배치 뒤에는 채널로 생각이됨
3. 이미 전처리 과정에서 채널을 더 해주기 때문에 이 부분 불필요한듯
4. 전처리 코드 -> 메모리에러...
5. 전처리 된 tFrecord에서 학습할 수 있도록 설계해보자

### 전처리 포함된 코드 일단 stop

In [11]:
## Making Read_fn

data_path = ct_set

def read_fn(file_reference, mode, new_size=256 ,params=None):
    
    
    ## file_refrences를 파일 path로 일단 생각하자
    
    for file_dict in file_reference:
        
        file_name = file_dict['name']
        file_image = file_dict['image']
        file_label = file_dict['label']
        
        
        ## 이미지 데이터를 로드하는 함수 인듯?

        
        t1 = nii_loader(data_path,file_image)
        lbl = nii_loader(data_path, file_label)
        
        
        ## whitening , 평균 0, 표준편차 1로 만들자
        
        ###### 나중에 추가하자!
        #t1 = whitening(t1)
        
        ## 재혁이 전처리
        
        print("="*10,t1.shape)
        
        t1 = pad3d(t1)
        t1 = image_preprocess(t1,new_size=new_size)
        
        
        # 4D Tensor with a dummy dimension for channels
        # ex (363, 512, 512) --> (362, 512, 512, 1)
        t1 = t1[...,np.newaxis]
        
        ### 
        print("Preprocessed Image Shape", t1.shape)
        
        
        ## prdeict mode 일 때는 label이 없다! 따라서 label 없는 값을 yield!
        if mode == tf.estimator.ModeKeys.PREDICT:
            yield{'features': {'x': t1},
                 'metadata':{
                     'subject_id': subgject_id,
                     'sitk': sitk_t1
                 }}
            
            
        ## Labels:

        
                ## 이미지 데이터를 로드하는 함수 인듯?

        #  lbl = image_preprocess(lbl,new_size=new_size,mask=True)

        lbl = image_preprocess(lbl,new_size=new_size)
        lbl = lbl[..., np.newaxis]
        y = lbl
        
        print("Preprocessed label Shape", y.shape)

        
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            
            # Insert augmentation here
            pass
        
#         if params['extract_examples']:
#             images = extract_random_example_array(
#                 t1,
#                 example_size=params['example_size'],
#                 n_examples=params['n_examples'])
            
#             # Loop the extracted image patches and yield
#             for e in range(params['n_examples']):
#                 yield {'features': {'x': images[e].astype(np.float32)},
#                        'labels': {'y': y.astype(np.int32)},
#                        'metadata': {
#                            'subject_id': subject_id,
#                            'sitk': sitk_t1}}
        
    
        ## Batch 1 이라고 할 때
#         else:
        yield {'features' : {'x': t1},
                'labels':{'y' : y}}
        

from dltk.io.abstract_reader import Reader

im_width = 256
im_height = 256
im_depth = 256


# Set up a data reader to handle the file i/o. 
reader_example_shapes = {'features': {'x': [im_width, im_height, im_depth,1]},
                         'labels': {'y': [im_width, im_height, im_depth,1]}}

reader = Reader(read_fn, {'features': {'x': tf.float32},
                          'labels': {'y': tf.int32}})

input_fn, qinit_hook = reader.get_inputs(file_references=total_list,
                                         mode=tf.estimator.ModeKeys.TRAIN,
                                         example_shapes=reader_example_shapes)
nn = tf.estimator.Estimator(model_fn=model_fn, 
                            model_dir=None, 
                            params={"learning_rate": 0.001})

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\NAKYIL~1\\AppData\\Local\\Temp\\tmpwt_zdkja', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002755BFB8C88>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [12]:
## padding 값에 음수?
## height weigth 문제 있는 듯

## 문제 1.
## 지금 shape 순서가 (H,W,D) 순서가 바뀜

## 2.
## Tensorshape이 원하는 데로 안 들어감 // 3D conv가 안 되고 있음

_ = nn.train(input_fn=input_fn, hooks=hooks + [qinit_hook], steps=2)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Init conv tensor shape (?, 128, 128, 128, 16)
INFO:tensorflow:Encoder at res_scale 1 tensor shape: (?, 64, 64, 64, 32)
INFO:tensorflow:Encoder at res_scale 2 tensor shape: (?, 32, 32, 32, 64)
INFO:tensorflow:Upsampling from (None, 32, 32, 32, 8) to [None, 64, 64, 64, 8]
INFO:tensorflow:Decoder at res_scale 1 tensor shape: (?, 64, 64, 64, 8)
INFO:tensorflow:Upsampling from (None, 64, 64, 64, 8) to [None, 128, 128, 128, 8]
INFO:tensorflow:Decoder at res_scale 0 tensor shape: (?, 128, 128, 128, 8)
INFO:tensorflow:Output tensor shape (?, 128, 128, 128, 8)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\NAKYIL~1\AppData\Local\Temp\tmpwt_zdkja\model.ckpt.
========== (512, 512, 363)
Array Shape (512, 512, 363)
height, Depth 512 363
pad_f

InvalidArgumentError: ValueError: Unable to create correctly shaped tuple from ((0, 0), (0, 0), (75, 74))
Traceback (most recent call last):

  File "C:\Users\Nakyilkim\Anaconda3\lib\site-packages\numpy\lib\arraypad.py", line 935, in _normalize_shape
    shape_arr = np.broadcast_to(shape_arr, (ndims, 2))

  File "C:\Users\Nakyilkim\Anaconda3\lib\site-packages\numpy\lib\stride_tricks.py", line 176, in broadcast_to
    return _broadcast_to(array, shape, subok=subok, readonly=True)

  File "C:\Users\Nakyilkim\Anaconda3\lib\site-packages\numpy\lib\stride_tricks.py", line 128, in _broadcast_to
    op_flags=[op_flag], itershape=shape, order='C')

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (3,2) and requested shape (0,2)


During handling of the above exception, another exception occurred:


Traceback (most recent call last):

  File "C:\Users\Nakyilkim\Anaconda3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 206, in __call__


In [16]:
################# Debugging

def pad3d(array):
    
    assert len(array.shape) == 3
    
    height = array.shape[0]
    depth = array.shape[2]
    
    print("Array Shape", array.shape)
    print("height, Depth",height,depth)
    
    if (height - depth) % 2 :
        pad_front = int((height + 1 - depth) / 2)
        pad_back = int((height - 1 - depth) / 2)
    else:
        pad_front = pad_back = int((height - depth) / 2)
        
    print("pad_front, pad_back",pad_front,pad_back)
    
    npad = ((0,0),(0,0),(pad_front,pad_back))
    array_padding = np.pad(array, npad, 'constant', constant_values=(0))
    array_padding[array_padding<0] = 0
    
    return array_padding

In [17]:
data_path = ct_set

file_dict = total_list[0]
file_name = file_dict['name']
file_image = file_dict['image']
file_label = file_dict['label']

file_list = []

for record in total_list:
    file_image = record['image']
    n1 = nii_loader(data_path,file_image)
    file_list.append(pad3d(n1.get_data()))


Array Shape (512, 512, 363)
height, Depth 512 363
pad_front, pad_back 75 74


MemoryError: 

In [15]:
file_list

[array([[[-1024., -1024., -1024., ..., -1024., -1024., -1024.],
         [-1024., -1024., -1024., ..., -1024., -1024., -1024.],
         [-1024., -1024., -1024., ..., -1024., -1024., -1024.],
         ...,
         [-1024., -1024., -1024., ..., -1024., -1024., -1024.],
         [-1024., -1024., -1024., ..., -1024., -1024., -1024.],
         [-1024., -1024., -1024., ..., -1024., -1024., -1024.]],
 
        [[-1024., -1024., -1024., ..., -1024., -1024., -1024.],
         [-1024., -1024., -1024., ..., -1024., -1024., -1024.],
         [-1024., -1024., -1024., ..., -1024., -1024., -1024.],
         ...,
         [-1024., -1024., -1024., ..., -1024., -1024., -1024.],
         [-1024., -1024., -1024., ..., -1024., -1024., -1024.],
         [-1024., -1024., -1024., ..., -1024., -1024., -1024.]],
 
        [[-1024., -1024., -1024., ..., -1024., -1024., -1024.],
         [-1024., -1024., -1024., ..., -1024., -1024., -1024.],
         [-1024., -1024., -1024., ..., -1024., -1024., -1024.],
      

In [17]:
x

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.